In [1]:
#this Jupyter notebook is intended to run locally (i.e. not in Azure)
#it prepares the raw csv files provided
#  master output (account and demographic information)
#  payment output (timeseries payment information for accounts)

#notebook creates wide, flat output file for training and validating ML model
#output will be all account and demographic info with 24 months of payment history (concatenated as columns)

#pandas is a python library
#  provides series class, 1D object, used here to work w dataframes
#  provides dataframe class, 2D object with spreadsheet like functionality
import pandas as pd
import numpy as np # for np.dateinterval64
from math import floor # for month roundings in dateinteraval



In [8]:
#open csv's as Pandas dataframes
#master will contain all account and demographic info
#payments will contain all payment history
master = pd.read_csv('master output.csv')
payments = pd.read_csv('payment output.csv')

#convert date columns to Pandas datetime datatypes (to assist with sorting)
master['dateplaced'] = pd.to_datetime(master['dateplaced'])
payments['DatePosted']= pd.to_datetime(payments['DatePosted'])

#sort both dataframes
master = master.sort_values(by=['customerid', 'accountid'])
payments = payments.sort_values(by=['customerid', 'accountid', 'DatePosted'])

#DEBUG
#payments.dtypes
#master.dtypes
print('done')


done


In [ ]:
output = pd.DataFrame()
record = 0

for index, row in master.iterrows():

    #get current account data    
    account_id = row['accountid']
    customer_id = row['customerid']

    
    #create blank payment history series
    pay_hist = pd.Series(dtype=object)
    for x in range(25):
        pay_hist = pay_hist.append(pd.Series({'Month'+str(x): 0}))

    #get payment rows
    account_payments = payments.loc[(payments['customerid'] == customer_id) & (payments['accountid']==account_id)]


    #if there are payments, fill in pay_hist 
    #only looking at first 24 months of payments
    for pay_index, pay_row in account_payments.iterrows():
        #get months between payment and account placement
        month = floor( (pay_row['DatePosted']-row['dateplaced']) / np.timedelta64(1, 'M') )
        if (month >= 0) and (month <= 24): #only look at first 24 months
            pay_hist['Month'+ str(month)] += pay_row['Collected'] #update pay history (will automatically subtract NSF)

    #calculate payment total and append to pay_hist
    pay_hist = pay_hist.append(pd.Series({'paytotal' : pay_hist.sum()}))

    #create output row and add to output data frame
    #row.append adds columns to series object row
    #output.append adds row to dataframe object output
    output = output.append(row.append(pay_hist), ignore_index=True)

    #DEBUG
    #print(output)
    record += 1
    print(record)
        
        

In [ ]:

#write to file and finish
output.to_csv('output.csv', index=True)
print('DONE')